In [ ]:
from os import path
import pandas as pd
import seaborn as sns; sns.set()
import numpy as np
from scipy.stats import linregress, pearsonr, spearmanr, kruskal
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from matplotlib.patches import Patch, Ellipse
from matplotlib.lines import Line2D
from matplotlib.gridspec import GridSpec
plt.style.use('seaborn-dark-palette')

In [ ]:
# Importing data
df = pd.read_csv("../data/metadata.csv")
df = df[~df['checkm_completeness'].isna()]
df = df[df['quality_filter']]

# Format columns
def fill_rank(row, rank='genus'):
    ranks = {
        "genus": 0,
        "species": 1,
        "strain": 2
    }
    label = ""
    if row['is_synecho']:
        if ranks[rank] < 2:
            label = str(row['label'].split("_")[ranks[rank]])
        if len(label) > 1:
            pass
        else:
            label = str(row[f'gtdb_{rank}'])
    else:
        label += str(row[f'gtdb_{rank}'])
    return label

# Formatting columns
df["gc"] = df["gc"] * 100
df['new_genus'] = df.apply(lambda row: 'Other cyanobacteria' if row['is_synecho'] == False else row['new_genus'], axis=1)

In [ ]:
markers = ['o', 'v', 'X', 'd', 's', 'P', 'D'] * 5

In [ ]:
# Plot function
def main2(data, data_, figsize=(15,15), axis_lim=((1,7), (30,70)), palette='deep', **kwargs):
    
    # Setting general attributes
    sns.set_palette(palette)
    markers = ['o', 'v', 'X', 'd', 's', 'P', 'D'] * 5 # other marker choices are: 'sDdPX'
    
    # Fig instance
    fig = plt.figure(figsize=figsize)
    gs = GridSpec(nrows=3, ncols=2, height_ratios=[1,1,1], wspace=0.05)

    # Data formatting
    data = data.copy()
    data['no_genus'] = data.groupby('new_genus')['new_genus'].transform('count').map(int, na_action="ignore")
    data = data.rename(columns={'new_genus': "Genus"})
    data["Genus"] = data["Genus"].apply(lambda s: r"${}$".format(s))
    data["Genus"] = data.apply(lambda row: row["Genus"] + f" ({str(row['no_genus'])})", axis=1)
    data = data.sort_values("no_genus", ascending=False)
    
    # First axis - scatterplot
    ax0 = fig.add_subplot(gs[:2, :])
    if axis_lim:
        ax0.set_xlim(axis_lim[0])
        ax0.set_ylim(axis_lim[1])
        
    ecotypes = {
    "$Parasynechococcus$ (53)": "Marine",
    "$Pseudosynechococcus$ (41)": "Marine",
    "$Synechospongium\ \mathrm {gen. nov.}$ (30)": "Marine",
    "$Vulcanococcus$ (5)": "Halotolerant",
    "$Lacustricoccus\ \mathrm {gen. nov.}$ (3)": "Freshwater",
    "$Regnicoccus$ (6)": "Halotolerant",
    "$Inmanicoccus$ (10)": "Marine",
    "$Cyanobium$ (7)": "Halotolerant",
    "$Synechococcus$ (6)": "Freshwater",
    "$Thermosynechococcus$ (6)": "Thermophilic",
    "$Limnothrix$ (14)": "Halotolerant",
    "$Leptococcus$ (10)": "Thermophilic",
    "$Brevicoccus$ (1)": "Marine",
    "$Stenotopis$ (1)": "Freshwater",
    "$Neosynechococcus$ (1)": "Peat bog",
    }

    ecotype_color = {
    "Marine": "#000080",
    "Freshwater": "#009000",
    "Thermophilic": "#900090",
    "Halotolerant": "#707030",
    "Peat bog": "k"
    }
    
    order = []
    for k, v in ecotypes.items():
        if v == "Marine":
            order.append(k)\
            
    for k, v in ecotypes.items():
        if v == "Halotolerant":
            order.append(k)
            
    for k, v in ecotypes.items():
        if v == "Freshwater":
            order.append(k)
            
    palette=palette=[ecotype_color[ecotypes[i]] for i in data["Genus"].value_counts().index]
    no_markers = len(data["Genus"].value_counts())

    sns.scatterplot(x='megabases', y='gc', data=data, alpha=0.7,
                    hue=data["Genus"], style=data["Genus"], hue_order=data["Genus"].value_counts().index,
                    s=100, ax=ax0, zorder=1, markers=markers[:no_markers], palette=palette)
    
    sns.scatterplot(x='megabases', y='gc', data=data_,
                        color='lightgray', alpha=0.6, s=100, ax=ax0, zorder=0)
        
    # Second axis - size plot
    palette = list(palette)[:-3] + ['#000080', '#009000', 'k']
    ax1 = fig.add_subplot(gs[2, 0])
    sns.boxplot(
        y="Genus",
        x="megabases",
        data=data,
        color='lightgray',
        boxprops=dict(alpha=.3),
        ax=ax1)
    sns.swarmplot(
        y="Genus",
        x="megabases",
        data=data,
        hue="Genus",
        palette=palette,
        alpha=0.7,
        zorder=0,
        ax=ax1)
    
    # Third axis - GC plot
    # GC plot
    ax2 = fig.add_subplot(gs[2, 1], sharey=ax1)
    sns.boxplot(
        y="Genus",
        x="gc",
        data=data,
        color='lightgray',
        boxprops=dict(alpha=.3),
        ax=ax2)
    sns.swarmplot(
        y="Genus",
        x="gc",
        data=data,
        hue="Genus",
        palette=palette,
        alpha=0.7,
        zorder=0,
        ax=ax2)
    
    # Hiding legends
    plt.setp(ax2.get_yticklabels(), visible=False)
    ax1.legend().remove()
    ax2.legend().remove()
    # ax1.grid(False)
    # ax2.grid(False)

    # Fixing labels
    ax0.set_xlabel("Size (Mbp)", )
    ax0.set_ylabel("GC content (%)", rotation=90, )
    ax1.set_xlabel("Size (Mbp)", )
    ax1.set_ylabel("")
    ax2.set_xlabel("GC content (%)", )
    ax2.set_ylabel("")
    
    for ax in (ax0, ax1, ax2):
        ax.patch.set_facecolor('#EEEEEE')
        ax.spines['bottom'].set_color('#EEEEEE')
        ax.spines['top'].set_color('#EEEEEE')
        ax.xaxis.label.set_color('#EEEEEE')
        ax.yaxis.label.set_color('#EEEEEE')
        ax.xaxis.label.set_size(16)
        ax.yaxis.label.set_size(16)                
        ax.tick_params(axis='x', colors='#EEEEEE', labelsize=16)
        ax.tick_params(axis='y', colors='#EEEEEE', labelsize=16)        
    fig.patch.set_alpha(0.0)
    # Other kwargs
    if 'legend_pos' in kwargs.keys():
        plt.legend(loc=kwargs['legend_pos'])
    
    if 'save' in kwargs.keys():
        plt.savefig(kwargs['save'], dpi=200, bbox_inches='tight')#, transparent=True)
    
    plt.show()

In [ ]:
# Our draft
df_ = df[df['quality_filter']].copy()

df_['no_genus'] = df_.groupby('new_genus')[
    'new_genus'].transform('count').map(int, na_action="ignore")

data = df_[(df_['is_synecho']) &
           ~(df_['new_genus'].isin(("GCA", "PCC-7336"))) &
           (df_['no_genus'] > 0) & 
           (df_['gtt_label'].str[1] != "X") &
           (~df_["new_genus"].isin(("Phormidesmis", "Leptovivax")))].copy()  # Remove non-SC genomes
data['new_genus'] = data['new_genus'].apply(lambda s: s + "\ \mathrm {gen. nov.}" if any(i in s for i in ("Lacustricoccus", "Synechospongium")) else s)
data_ = df_[~df_['is_synecho']].copy()

models = []

for dataframe in (data, data_):
    model = linregress(dataframe['checkm_completeness'], dataframe['megabases'])
    models.append(model)
    dataframe['megabases'] = dataframe.apply(lambda row: 100 *
                                   row['megabases'] / row['checkm_completeness'] if
                                   row['checkm_completeness'] < 100 else row['megabases'], axis=1)

main2(data=data, data_=data_, figsize=(17.5,17.5), axis_lim=((1.5,5.5), (45, 70)), save="ProSynFest_GC_ticks.png")

## Supplementary Figure - Without adjusted size values

In [ ]:
# Our draft
df_ = df[df['quality_filter']].copy()

df_['no_genus'] = df_.groupby('new_genus')[
    'new_genus'].transform('count').map(int, na_action="ignore")

data = df_[(df_['is_synecho']) &
           ~(df_['new_genus'].isin(("GCA", "PCC-7336"))) &
           (df_['no_genus'] > 0) & 
           (df_['gtt_label'].str[1] != "X") &
           (~df_["new_genus"].isin(("Phormidesmis", "Leptovivax")))].copy()
data['new_genus'] = data['new_genus'].apply(lambda s: s + "\ \mathrm {gen. nov.}" if any(i in s for i in ("Lacustricoccus", "Synechospongium")) else s)
data_ = df_[~df_['is_synecho']].copy()

models = []

# for dataframe in (data, data_):
#     model = linregress(dataframe['checkm_completeness'], dataframe['megabases'])
#     models.append(model)
#     dataframe['megabases'] = dataframe.apply(lambda row: 100 *
#                                    row['megabases'] / row['checkm_completeness'] if
#                                    row['checkm_completeness'] < 100 else row['megabases'], axis=1)

main2(data=data, data_=data_, figsize=(17.5,17.5), axis_lim=((0.5,5.5), (45, 70)), save="../data/plots/final/Supp_Figure_3_review.pdf")